In [27]:
import pandas as pd
import hddCRP.dataLoader as dl

In [28]:
subject = "C2"
df = pd.read_csv(f"data/raw/Data_DIO_phase2_{subject} copy.csv", index_col=0)
df.sort_values(["session", "on_time"], inplace=True)
df = df[["session", "Rewarded", "well_id", "on_time"]]
df["session_id"] = df.groupby(df["session"]).ngroup()
df["trial_number"] = df.groupby("session_id")["well_id"].transform(lambda rr : (rr != rr.shift()).cumsum() - 1)

def get_first_well_entry(pp):
    xx = pp.iloc[0]
    xx["Rewarded"] = pd.Series.any(pp["Rewarded"])
    return xx
df = df.groupby(["session","trial_number"],as_index=False).apply(get_first_well_entry)
df.sort_values(["session", "trial_number"], inplace=True)


def turns(well_id):
    well_id -= 1
    ss = (well_id == (well_id.shift() + 2) % 4);
    ll = (well_id == (well_id.shift() + 1) % 4);
    rr = (well_id == (well_id.shift() - 1) % 4);
    well_id = well_id.astype(str)
    well_id[:] = "None"
    well_id[ss] = "Straight"
    well_id[ll] = "Left"
    well_id[rr] = "Right"
    return well_id

df["turn"] = df.groupby(["session"],as_index=False)["well_id"].transform(turns)
df["turn_idx"] = df["turn"].map({"None" : pd.NA, "Straight" : 1, "Left" : 0, "Right" : 2})
df.head()


,session,Rewarded,well_id,on_time,session_id,trial_number,turn,turn_idx
0,20210206_03-0,False,3,4275048,0,0,None,<NA>
1,20210206_03-0,False,1,4907287,0,1,Straight,1
2,20210206_03-0,False,4,5632117,0,2,Right,2
3,20210206_03-0,False,2,5805261,0,3,Straight,1
4,20210206_03-0,False,1,6440635,0,4,Right,2


In [29]:
grouped = df.groupby(["session_id"])

seqs = []
locs = []
for session, group in grouped:
    tts = group["turn_idx"]
    wws = group["well_id"]
    tts = tts[pd.notna(tts)].values
    seqs.append(list(tts))
    locs.append(list(wws))
# compare to the file I got from Jai
seqs_from_processed = dl.get_phase2(subject);
    

print(f"locations\nnew session 0: {locs[0]}")
print(f"turns\nnew session 0: {seqs[0]}")
print(f"old session 0: {seqs_from_processed[0]}")
print(f"trimmed\nold session 0: {seqs_from_processed[0][2:-3]}")

locations
new session 0: [3, 1, 4, 2, 1, 3, 2, 4, 1, 2, 4, 3, 1, 2, 4, 1, 3, 2, 4, 1, 2, 4, 1, 3, 1, 4, 2, 1, 3, 1, 4, 1, 3, 2]
turns
new session 0: [1, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2, 0, 1, 2]
old session 0: [1, 0, 1, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2, 0, 1, 2, 0, 2, 2]
trimmed
old session 0: [1, 2, 1, 2, 1, 2, 1, 0, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2, 0, 1, 2]
